# Simple database queries

This notebook demonstrates the core tables in the DIAS data base and lists a number of queries that can be used to select data from the database.

The DIAS database is PostgreSQL with the Postgis extension installed. Postgis is required to handle spatial data features and perform spatial queries. The database server runs inside a docker container which is based on the standard mdillon/postgis image. The server is accessible from any VM in the DIAS cluster.

## Database tables
Three tables contain the core data we need in this tutorial. There are:

 - __dias_catalogue__: contains the metadata for all CARD images that are processed on the DIAS; (Columns: id, obstime, reference, sensor, card, status, footprint)
 - __parcels_2018__ : contains the parcel vectors of the working area, with their attributes; (e.g.: ogc_fid, id, parcel_idcrop_id, crop_type, ha, wkb_geometry)
 - __parcels_2018_signatures_CD__(e.g.: es2018_signatures_bs): contains extracted statistics for the combination of parcel vectors and CARD images (CD=bs for S1 backscattering coefficients, CD=c6 for S1 6 day coherence, CD=s2 for Sentinel-2 Level 2A)
 - __aois__: (optional) contains the monitoring areas of the member state.
 
(spatial_ref_sys is a standard table in a postgis data base that contains spatial projection definitions)

In [1]:
# import required libraries for this Notebook
import psycopg2
import pandas as pd
from datetime import datetime
from ipywidgets import widgets


In [2]:
# Set the database credentials

DB_HOST = '172.17.0.2'
DB_NAME = 'postgres'
DB_USER = 'postgres'
DB_PORT = '5432'
DB_PASS = ''
DB_SCHE = 'public'

conn_str = f"host={DB_HOST} dbname={DB_NAME} user={DB_USER} port={DB_PORT} password={DB_PASS}"

In [3]:
# Connect to the database
conn = psycopg2.connect(conn_str)

In [4]:
# Get a sample of the first rows of CARD images table ({dias_catalog}) without the use of pandas.

# Create a "cursor" to allow Python code to execute PostgreSQL commands in a database session.
# This is needed only if you don't use pandas

getSampleList = f"""
    SELECT * FROM dias_catalogue
    LIMIT 2;
"""

cur = conn.cursor()
cur.execute(getSampleList)

for rows in cur:
    print(rows)


(1772, datetime.datetime(2020, 12, 20, 11, 23, 59, 24000), 'S2B_MSIL2A_20201220T112359_N0214_R037_T29SMD_20201220T132403.SAFE', '2B', 's2', 'Obsolete', '0103000020E6100000010000000C000000E2B034F0A3EA23C0B200AD4EC6604340E6EB32FCA7E323C029CDD5ECBD664340516859F78FCD23C02F7A590A98794340CBD765F84FB723C0AFE56610718C43401842CEFBFFA023C072F7E8F64A9F43400AA2EE03908A23C0A23AC4F323B2434037FA980F087423C056C5D04FFDC443408D60E3FA775D23C06C6A9117D7D743403D0E83F92B5423C0098B5F3093DF43400169FF03ACC521C0525A91F206E04340919BE1067CC621C029E3D73C62614340E2B034F0A3EA23C0B200AD4EC6604340')
(1771, datetime.datetime(2020, 12, 20, 11, 23, 59, 24000), 'S2B_MSIL2A_20201220T112359_N0214_R037_T29TPE_20201220T132403.SAFE', '2B', 's2', 'Obsolete', '0103000020E61000000100000005000000A738B302E8441FC0278E80CB8852444037FA980F08141AC09EC53C1CE64F4440DA1D520C90381AC0B6E7803C64D143404A9869FB57561FC0263C89DAEFD34340A738B302E8441FC0278E80CB88524440')


## Converting data to pandas dataframe

Pandas takes data (like a CSV or TSV file, or a SQL database) and creates a Python object with rows and columns called data frame that looks very similar to table in a statistical software (like Excel or SPSS, with some similarities to R). This is easier to work with in comparison to working with lists and/or dictionaries through for loops or list comprehensions.

In [16]:
# Get the tables list
getTablesList = f"""
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';
"""

cur = conn.cursor()
cur.execute(getSampleList)

for rows in cur:
    print(rows)


('public', 'spatial_ref_sys', 'postgres', None, True, False, False, False)
('topology', 'topology', 'postgres', None, True, False, True, False)
('topology', 'layer', 'postgres', None, True, False, True, False)
('public', 'dk2018_dias_catalogue', 'postgres', None, False, False, False, False)
('fr', 'parcels_2020', 'postgres', None, False, False, False, False)
('public', 'cat2020', 'postgres', None, True, False, False, False)
('public', 'dk2018_s2_signatures', 'postgres', None, False, False, False, False)
('public', 'dias_catalogue', 'postgres', None, False, False, False, False)
('fr', 'sigs_2020', 'postgres', None, False, False, False, False)
('fr', 'hists_2020', 'postgres', None, False, False, False, False)
('public', 'cat2020_s2_signatures', 'postgres', None, True, False, False, False)
('public', 'dk2018_bs_signatures', 'postgres', None, False, False, False, False)
('public', 'aois', 'postgres', None, True, False, False, False)
('public', 'cat2020_dias_catalogue', 'postgres', None, Tr

In [23]:
# Get the first rows of the selected table within pandas dataframe:

getSampleList = f"""
    SELECT * FROM dias_catalogue
    LIMIT 2
"""

# Read result set into a pandas dataframe
df_SampleList = pd.read_sql_query(getSampleList,  conn)
df_SampleList

,id,obstime,reference,sensor,card,status,footprint
0,1772,2020-12-20 11:23:59.024,S2B_MSIL2A_20201220T112359_N0214_R037_T29SMD_2...,2B,s2,Obsolete,0103000020E6100000010000000C000000E2B034F0A3EA...
1,1771,2020-12-20 11:23:59.024,S2B_MSIL2A_20201220T112359_N0214_R037_T29TPE_2...,2B,s2,Obsolete,0103000020E61000000100000005000000A738B302E844...


In [24]:
# Get exact count of rows for the selected table.

pgq_exact_count = f"""
    SELECT count(*) AS exact_count
    FROM public.dias_catalogue;
    """

df_exact_count = pd.read_sql_query(pgq_exact_count, conn)
df_exact_count

,exact_count
0,29817


In [25]:
# Check parcel size distribution:

getSizeDistributionSql = f"""
    SELECT count(*) total, sum(st_area(wkb_geometry))/10000.0 ha
    FROM cat2020
    WHERE st_area(wkb_geometry) < 3000.0;
"""

df_sizeDist = pd.read_sql_query(getSizeDistributionSql, conn)
df_sizeDist

,total,ha
0,493,86.131987


In [31]:
# Get signatures for a parcel from database in a dataframe:

parcel_id = 1000

getSignaturesSql = f"""
    SELECT * FROM cat2020_s2_signatures
    WHERE pid={parcel_id} LIMIT 4;
"""

df_signatures = pd.read_sql_query(getSignaturesSql,  conn)
df_signatures

,pid,obsid,band,count,mean,std,min,max,p25,p50,p75
0,1000.0,734,B11,229.0,1658.0742,206.30470,967.0,2281.0,1530.0,1660.0,1786.0
1,1000.0,734,B12,229.0,1194.9912,163.90689,673.0,1745.0,1090.0,1184.0,1296.0
2,1000.0,730,B11,229.0,1601.5459,259.50046,339.0,2349.0,1458.0,1600.0,1737.0
3,1000.0,730,B12,229.0,1164.0393,208.87700,210.0,1861.0,1041.0,1151.0,1259.0


In [ ]:
# Select an Area of Interest:

aois_list = database.table_data(aois_table, 1)['name'].tolist() # Get the tables as python list
aoi_name = widgets.Dropdown(options=aois_list, description="AOI: ")
print("Select the aoi to display information.")
aoi_name

# Use with: aoi_name.value

In [34]:
# Get some rows of CARD images metadata:
aoi_name = 'parcels_2020'

getMetadataSql = f"""
    SELECT reference, obstime, st_astext(footprint)
    FROM dias_catalogue, aois
    WHERE footprint && wkb_geometry
    And name = '{aoi_name}'
    And st_intersects(footprint, wkb_geometry)
    ORDER by obstime limit 4;
"""

# Read result set into a pandas dataframe
df_metadata = pd.read_sql_query(getMetadataSql, conn)
df_metadata

,reference,obstime,st_astext


In [36]:
# Get the total number of CARD images for the selected aoi:

getCARDCountSql = f"""
    SELECT card, count(*) FROM dias_catalogue, aois
    WHERE footprint && wkb_geometry
    And name = '{aoi_name}'
    GROUP by card;
"""

# Read result set into a pandas dataframe
df_card_count = pd.read_sql_query(getCARDCountSql, conn)
df_card_count

,card,count


In [37]:
# Get the total number of CARD images for the selected aoi and date sorted by sensor, card type and status:
start_date = '01-01-2020'
end_date = '05-05-2020'


getRowCountSql = f"""
    SELECT sensor, card, status, count(*)
    FROM dias_catalogue, aois
    WHERE footprint && wkb_geometry
    And name = '{aoi_name}'
    And obstime between '{start_date}' and '{end_date}'
    GROUP by sensor, card, status
    ORDER by sensor, card, status
"""

# Read result set into a pandas dataframe
df_rowcount = pd.read_sql_query(getRowCountSql, conn)
df_rowcount

,sensor,card,status,count


**The following examples need the field "crop type", in case this field is in a different table follow the examples given in the notebook "" for how to join the tables.**

In [ ]:
# Get a time series for a specific parcel:

crop_type_column = ''
parcels_table = ''
parcel_id = 441210
bs_signatures = 'signatures_2020_bs'

getTimeSeriesSql = f"""
    SELECT obstime, band, ogc_fid::int, {crop_type_column}, 
    mean, std, count, st_area(wkb_geometry)/10000 As ha 
    FROM dias_catalogue dc, {parcels_table} parcels, 
    {bs_signatures} bs
    WHERE dc.id = bs.obsid 
    And parcels.ogc_fid = bs.pid
    And parcels.ogc_fid = {parcel_id}
    ORDER by obstime asc
    LIMIT 2;
"""

# Read result set into a pandas dataframe
df_timeseries = pd.read_sql_query(getTimeSeriesSql,  conn)
df_timeseries

In [ ]:
# Format the geometry into something more readable, as do some on-the-fly calculation
# and sample from the table randomly:

getParcelSpecs2Sql = f"""
    SELECT ogc_fid::int as pid, {crop_type_column},
    ST_astext(wkb_geometry) as parcel, 
    ST_area(wkb_geometry)/10000 as ha
    FROM {parcels_table} 
    tablesample system (1.0)  -- This select a 1% random sample, so should give different results for each run
    WHERE st_area(wkb_geometry) > 10000.0   -- This ensures we only select parcels > 1 ha
    LIMIT 2;
"""

df_parcelspecs = pd.read_sql_query(getParcelSpecs2Sql, conn)
df_parcelspecs

In [ ]:
# Get parcels SRID.

pgq_srid = f"""
    SELECT ST_SRID(wkb_geometry) FROM {parcels_table} LIMIT 1;
    """

df_srid = pd.read_sql_query(pgq_srid, conn)
srid = df_srid['st_srid'][0]
print("SRID: ", srid)

In [ ]:
# Check how many parcels are in a particular aoi:
# Note: this query may take some time.
srid = ''

getParcelsPerAoiSql = f"""
    SELECT name, count(parcels.ogc_fid) total, sum(st_area(parcels.wkb_geometry))/10000.0 ha
    FROM {parcels_table} parcels, aois
    WHERE parcels.wkb_geometry && st_transform(aois.wkb_geometry, {srid})
    GROUP by name;
"""

df_ParcelsPerAoi = pd.read_sql_query(getParcelsPerAoiSql, conn)
df_ParcelsPerAoi

In [ ]:
#(OPTIONAL) Export dataframe to csv file (Change 'df' with the name of the dataframe to be exported).
df.to_csv(config_ui.folder_data + "/dataframe_01.csv")

In [ ]:
# Close database connection
conn.close()